In [1]:
import pandas as pd
from datetime import date
import numpy as np
from dateutil.parser import parse 

In [2]:
order_data=pd.read_excel('唯衣POC.xlsx',sheet_name=0,header=0)#订单表
item_info=pd.read_excel('唯衣POC.xlsx',sheet_name=2,header=1)#商品信息表
item_id=pd.read_excel('唯衣POC.xlsx',sheet_name=3,header=0)#条码id表
order_data=order_data.drop(0)
item_id=item_id.drop(0)

In [3]:
user_item=pd.merge(order_data,item_id,left_on=['订单所含商品条码ID'],right_on=['条码id'],how='left')

In [4]:
user_item=user_item.drop(['订单总金额','订单所含商品单价','条码'],axis=1)

In [5]:
item_info=item_info[['ID','款号','年份','季节']]

In [6]:
order_item_info=pd.merge(user_item,item_info,left_on=['款号'],right_on=['款号'],how='left')

In [7]:
order_item_info['订单时间']=[pd.Period(parse(str(x)),freq='M') for x in order_item_info['订单时间']]

In [8]:
date_map={'春装':'01','夏装':'04','秋装':'07','冬装':'10','全年':'02',np.nan:np.nan}#季节月份对应表
order_item_info['衣服月份']=[ date_map[x] for x in order_item_info['季节']]

In [9]:
order_item_info['衣服上市时间']=np.nan
order_item_info['购买时差']=0

In [18]:
order_item_info['年份'].unique()

array(['2015年', '2016年', '2014年', '2013年', '2012年', 'nan', '2017年'],
      dtype=object)

In [17]:
order_item_info['年份']=order_item_info['年份'].astype('str')

In [ ]:
for index, row in order_item_info.iterrows():    
    if row['年份']!='nan':                  
            y=row['年份'][:4]
            m=row['衣服月份']        
            row['衣服上市时间']=pd.Period(y+'-'+m,freq='M')     

In [33]:
order_item_info['购买时差']=order_item_info['订单时间']-order_item_info['衣服上市时间']

In [40]:
len(order_item_info[order_item_info['购买时差']>-100])

3222

In [41]:
pd.Period('2017-01',freq='M') 

Period('2017-01', 'M')

In [ ]:
len(order_item_info[order_item_info['购买时差']!=0])

In [ ]:
len(np.where(np.isnan(order_item_info['ID']))[0])

In [ ]:
len(user_item['款号id'])